### Idea

The idea here is to trim the sequence up to the cut on both reads and then do a global alignment end-to-end agasint the human genome.

The idea here is to take the files that are 'trimmed' and 'correct priming' checking. Then to do another round of trimming to get down to the cut site. I need to make another folder for 'trimmed_to_cut'

For Tn5 setup the 'Read 1' contains the sequence away from the gene specific primer. So it is the read we should concentration on if we don't do paired end.

For LAM the 'Read 2' is the the sequence distal to the gene specific primer

In [1]:
from __future__ import print_function
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pylab
import pandas as pd
import numpy as np
import os
import sys
import gzip
import itertools
import operator
import subprocess
import twobitreader
from Bio.Alphabet import IUPAC
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import pysam

#not sure if I need these
class Error(Exception):
    """Base class for exceptions in this module."""
    pass

class StrandError(Error):
    """Exception raised for errors in the strand information.
    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """
    def __init__(self, message):
        self.message = message

class ReactionTypeError(Error):
    """Exception raised for errors in the reaction type to be processed.
    Attributes:
        expression -- input expression in which the error occurred
        message -- explanation of the error
    """
    def __init__(self, message):
        self.message = message



## Inputs
#### next few window are different inputs to make it run well

In [2]:
#things taken from original uditas args

check_plasmid_insertions = 1
ncpu = 4
window_size = 15
amplicon_window_around_cut = 1000
min_MAPQ = 5
min_AS = -180
process_AMP_seq_run = 0 #off



In [83]:
#This is for the sequence to the breaksite for the pipeline of genomic alignment 

print('this is for polb fwd. row 0 in sheet')
#seq_primer_to_breaksite = 'TGGTTTTATTTCACCCCATGATAGTAATTATATCACTTCTGATCTGTTAAGAATAGACCTTTTAAAAGTATTGGATAACTTAGAGATGAGACATCTTCAGTTACTCTGTTATTCACCTATTACTCCTTAGGTTACTTGTGAATAATTTTGTGTGGGTCA'


this is for polb fwd. row 0 in sheet


In [40]:
#Directory

directory = '/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_BCL2Fastq_only'

#this is a minimal directory of only 300 files for quick debugging
#directory = '/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_BCL2Fastq_only/10000Reads'

print(directory)

/media/edanner/NewUbuntuSpace/Workspace/LinearAmp/Sequence2_191129_MN00157_0047_A000H2GWGF/P_Eric4_BCL2Fastq_only


In [41]:
# Function to make the 'amplicon_info' list. Taking the line of our experiment csv file

def get_csv_data(dir_sample, line_of_data_from_csv):
    sample_info_filename = os.path.join(dir_sample, 'sample_info.csv')
    experiments = pd.read_csv(sample_info_filename)
    return experiments.loc[line_of_data_from_csv]


#the amplicon info is related to the line on the csv file. It is indexed from 0. For PolbF we use 0.
#it is on my extera space so need to make sure that is mounted. (first thing to check if it throws and error)

#0 is because this polbFwd is in the first line. Need to change.
amplicon_info = get_csv_data(directory, 0)

#1 is because this polbRev 
#amplicon_info = get_csv_data(directory, 1)

#2 is because this mcherry_rev
#amplicon_info = get_csv_data(directory, 2)

#3 is because this mcherry_fwd
#amplicon_info = get_csv_data(directory, 3)


###########check waht is printed below #######
amplicon_info

NGS_req-ID                                                                     A000H2GWGF
name                                                                        Tn5_Pol50_pbF
Sample                                                                               Polb
description                                   MiniSeq_K562_PolbHIROS_50per_unsorted_polbF
Control sample (Y/N)                                                                    N
Notes                                                                                 NaN
Dilution                                                                              NaN
Cell name_type                                                                       K562
UMI_Len                                                                        NNNNNNNNNN
IndexI7Primer                                                                      prE368
I7_Index_ID                                                          P7_N701_SBS12nextera
index_I1  

In [42]:
# Assign the file_genome_2bit location. This is needed for pulling sequence from the referene genome by location
assembly = amplicon_info['genome']
file_genome_2bit = os.path.join('/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes', assembly + '.2bit')
print(file_genome_2bit)

# BOWTIE2_INDEXES are needed for global alignments
#not sure if this will work
#normally in bash: export BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes
#check in bash: > ECHO $GENOMES_2BIT

#Needs to have a bowtie build in which the targeting vector is included!!

%env BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes

#check the env variables
#%env

/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes/hg38.2bit
env: BOWTIE2_INDEXES=/media/edanner/NewUbuntuSpace/Workspace/Ref_Genomes


### Functions to process the reads

In [59]:
#these are copied and unchanged from the Uditas v1 software

################################################################################
# Open .fastq or .fastq.gz files for reading
################################################################################
def open_fastq_or_gz(filename):
    if filename.endswith(".fastq") and os.access(filename, os.F_OK):
        return open(filename, "rU")
    elif filename.endswith(".fastq.gz") and os.access(filename, os.F_OK):
        return gzip.open(filename, "rb")
    elif filename.endswith(".fastq") and os.access(filename + ".gz", os.F_OK):
        return gzip.open(filename + ".gz", "rb")
    elif filename.endswith(".fastq.gz") and os.access(filename[:-3], os.F_OK):
        return open(filename[:-3], "rU")
    raise IOError("Unknown file: " + filename)

################################################################################
# Hamming distance
# From http://code.activestate.com/recipes/499304-hamming-distance/
################################################################################
def hamm_dist(str1, str2):
    assert len(str1) == len(str2)
    ne = operator.ne
    return sum(itertools.imap(ne, str1, str2))

################################################################################
# Select closest barcode with a maximum number of mismatches
# By default it returns barcodes with a maximum of n_max_mismatches=2 mismatches
################################################################################
def select_barcode(seq, barcode_list, n_max_mismatches=1):
    # This compares with all barcodes and selects the one with the smallest hamming distance
    # Before calling this function check if the sequence is already a barcode
    matched_barcodes = list()
    distances = list()
    for barcode in barcode_list:
        h_d = hamm_dist(seq, barcode)
        if h_d <= n_max_mismatches:
            matched_barcodes.append(barcode)
            distances.append(h_d)
    indices = [i for i, x in enumerate(distances) if x == min(distances)]
    return [matched_barcodes[i] for i in indices]


################################################################################
# Mask sequence by quality score
################################################################################
def mask(seq, qual, min_qual=12):

    return "".join((b if (ord(q) - 33) >= min_qual else "N") for b, q in itertools.izip(seq, qual))


################################################################################
# get the reverse-complement DNA sequence
################################################################################
def reverse_complement(seq):
    seq_dict = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G', 'N': 'N', 'a': 't', 't': 'a', 'g': 'c', 'c': 'g'}
    return "".join([seq_dict[base] for base in reversed(seq)])


################################################################################
# Create umi dict
################################################################################
def create_umi_dict(filename):

    umi_file = open_fastq_or_gz(filename)

    umi_dict = dict()

    umi_reads = itertools.izip(umi_file)

    for header_umi in umi_reads:

        seq_umi = umi_reads.next()
        umi_reads.next()
        qual_umi = umi_reads.next()
        umi_dict[header_umi[0].split()[0][1:]] = [seq_umi[0].rstrip(), qual_umi[0].rstrip()]

    return umi_dict



################################################################################
# create list of output files
#I added a bit to allow for the function that makes a fastq of the correctly primed targets
#I  need to add to make  mishas umi file
################################################################################
def create_filename(dir_sample, N7, N5, filetype):
    main_folder = os.path.join(dir_sample, N7 + '_' + N5)
    if filetype == 'mainfolder':
        return main_folder
    elif filetype == 'amplicons':
        return os.path.join(main_folder, 'amplicons')
    elif filetype == 'R1fastq':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.fastq')
    elif filetype == 'R1fastqgz':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.fastq.gz')
    elif filetype == 'R2fastq':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.fastq')
    elif filetype == 'R2fastqgz':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.fastq.gz')
   
    ### Added to make LAM cleanup after extracting UMI and cleaving adapter
    elif filetype == 'R1fastq_LAM':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.LAM.fastq')
    elif filetype == 'R1fastqgz_LAM':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.LAM.fastq.gz')
    elif filetype == 'R2fastq_LAM':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.LAM.fastq')
    elif filetype == 'R2fastqgz_LAM':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.LAM.fastq.gz')
    
    #### I added these as the sequences that were correctly primed
    elif filetype == 'R1fastq_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.CorrPrime.fastq')
    elif filetype == 'R1fastqgz_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R1.CorrPrime.fastq.gz')
    elif filetype == 'R2fastq_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.CorrPrime.fastq')
    elif filetype == 'R2fastqgz_CorrPrime':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_R2.CorrPrime.fastq.gz')
    #####
    
    elif filetype == 'umifastq':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_umi.fastq')
    elif filetype == 'umifastqgz':
        return os.path.join(main_folder, 'fastq_files', N7 + '_' + N5 + '_umi.fastq.gz')
    
    ### Added these for trimming to breaksite
    elif filetype == 'R1trimmed2break':
        return os.path.join(main_folder, 'cutadapt_files_to_breaksite', N7 + '_' + N5 + '_R1.trimmed2break.fastq.gz')
    elif filetype == 'R2trimmed2break':
        return os.path.join(main_folder, 'cutadapt_files_to_breaksite', N7 + '_' + N5 + '_R2.trimmed2break.fastq.gz')
    elif filetype == 'trimmed_report2break':
        return os.path.join(main_folder, 'cutadapt_files_to_breaksite', N7 + '_' + N5 + '.trimmed.report.txt')
    # added. for aligning to amplicons first
    elif filetype == 'break_trimmed_sam_amplicons':         
        return os.path.join(main_folder, 'break_trimmed_sam_amplicon_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'break_trimmed_sam_report_amplicons':
        return os.path.join(main_folder, 'break_trimmed_sam_amplicon_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'break_trimmed_bam_amplicons':
        return os.path.join(main_folder, 'break_trimmed_bam_amplicon_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'break_trimmed_sorted_bam_amplicons':
        return os.path.join(main_folder, 'break_trimmed_bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'break_trimmed_sorted_bai_amplicons':
        return os.path.join(main_folder, 'break_trimmed_bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'break_trimmed_unmapped_bam_amplicons':
        return os.path.join(main_folder, 'break_trimmed_bam_amplicon_files', N7 + '_' + N5 + '_amplicons_unmapped.bam')
    elif filetype == 'break_trimmed_qsorted_unmapped_bam_amplicons':
        return os.path.join(main_folder, 'break_trimmed_bam_amplicon_files', N7 + '_' + N5 + '_qsorted_amplicons_unmapped.bam')
    elif filetype == 'break_trimmed_unmapped_amplicons_R1fastq':
        return os.path.join(main_folder, 'break_trimmed_amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq')
    elif filetype == 'break_trimmed_unmapped_amplicons_R2fastq':
        return os.path.join(main_folder, 'break_trimmed_amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq')
    elif filetype == 'break_trimmed_unmapped_amplicons_R1fastqgz':
        return os.path.join(main_folder, 'break_trimmed_amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq.gz')
    elif filetype == 'break_trimmed_unmapped_amplicons_R2fastqgz':
        return os.path.join(main_folder, 'break_trimmed_amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq.gz')
    elif filetype == 'break_trimmed_unmapped_amplicons_report':
        return os.path.join(main_folder, 'break_trimmed_amplicons_unmapped_fastq_files', N7 + '_' + N5 + '.unmapped.report.txt')
    elif filetype == 'break_trimmed_sam_genome_global':
        return os.path.join(main_folder, 'break_trimmed_sam_genome_global_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'break_trimmed_sam_report_genome_global':
        return os.path.join(main_folder, 'break_trimmed_sam_genome_global_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'break_trimmed_bam_genome_global':
        return os.path.join(main_folder, 'break_trimmed_bam_genome_global_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'break_trimmed_sorted_bam_genome_global':
        return os.path.join(main_folder, 'break_trimmed_bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'break_trimmed_sorted_bai_genome_global':
        return os.path.join(main_folder, 'break_trimmed_bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'break_trimmed_sam_genome_local':
        return os.path.join(main_folder, 'break_trimmed_sam_genome_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'break_trimmed_sam_report_genome_local':
        return os.path.join(main_folder, 'break_trimmed_sam_genome_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'break_trimmed_bam_genome_local':
        return os.path.join(main_folder, 'break_trimmed_bam_genome_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'break_trimmed_sorted_bam_genome_local':
        return os.path.join(main_folder, 'break_trimmed_bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'break_trimmed_sorted_bai_genome_local':
        return os.path.join(main_folder, 'break_trimmed_bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    ###############################################
    
    elif filetype == 'R1trimmed':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '_R1.trimmed.fastq.gz')
    elif filetype == 'R2trimmed':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '_R2.trimmed.fastq.gz')
    elif filetype == 'trimmed_report':
        return os.path.join(main_folder, 'cutadapt_files', N7 + '_' + N5 + '.trimmed.report.txt')
    elif filetype == 'sam_genome_local':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_genome_local':
        return os.path.join(main_folder, 'sam_genome_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_genome_local':
        return os.path.join(main_folder, 'bam_genome_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'sam_plasmid_local':
        return os.path.join(main_folder, 'sam_plasmid_local_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_plasmid_local':
        return os.path.join(main_folder, 'sam_plasmid_local_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'unmapped_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '_unmapped.bam')
    elif filetype == 'qsorted_unmapped_bam_plasmid_local':
        return os.path.join(main_folder, 'bam_plasmid_local_files', N7 + '_' + N5 + '_qsorted_unmapped.bam')
    elif filetype == 'unmapped_plasmid_R1fastq':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R1.fastq')
    elif filetype == 'unmapped_plasmid_R2fastq':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R2.fastq')
    elif filetype == 'unmapped_plasmid_R1fastqgz':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R1.fastq.gz')
    elif filetype == 'unmapped_plasmid_R2fastqgz':
        return os.path.join(main_folder, 'plasmid_unmapped_fastq_files', N7 + '_' + N5 + '_plasmid_unmapped_R2.fastq.gz')
    elif filetype == 'sam_amplicons':
        return os.path.join(main_folder, 'sam_amplicon_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_amplicons':
        return os.path.join(main_folder, 'sam_amplicon_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'unmapped_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '_amplicons_unmapped.bam')
    elif filetype == 'qsorted_unmapped_bam_amplicons':
        return os.path.join(main_folder, 'bam_amplicon_files', N7 + '_' + N5 + '_qsorted_amplicons_unmapped.bam')
    elif filetype == 'unmapped_amplicons_R1fastq':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq')
    elif filetype == 'unmapped_amplicons_R2fastq':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq')
    elif filetype == 'unmapped_amplicons_R1fastqgz':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files',
                            N7 + '_' + N5 + '_amplicons_unmapped_R1.fastq.gz')
    elif filetype == 'unmapped_amplicons_R2fastqgz':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files',
                            N7 + '_' + N5 + '_amplicons_unmapped_R2.fastq.gz')
    elif filetype == 'unmapped_amplicons_report':
        return os.path.join(main_folder, 'amplicons_unmapped_fastq_files', N7 + '_' + N5 + '.unmapped.report.txt')
    elif filetype == 'sam_genome_global':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '.sam')
    elif filetype == 'sam_report_genome_global':
        return os.path.join(main_folder, 'sam_genome_global_files', N7 + '_' + N5 + '.sam.report.txt')
    elif filetype == 'bam_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.bam')
    elif filetype == 'sorted_bam_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam')
    elif filetype == 'sorted_bai_genome_global':
        return os.path.join(main_folder, 'bam_genome_global_files', N7 + '_' + N5 + '.sorted.bam.bai')
    elif filetype == 'results_amplicons':
        return os.path.join(main_folder, 'results', N7 + '_' + N5)  # We will append the window size later
    elif filetype == 'results_plasmid':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_plasmid.xlsx')
    elif filetype == 'results_all_amplicons':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_all_amplicons.xlsx')
    elif filetype == 'results_genomewide':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_results_genomewide.xlsx')
    elif filetype == 'summary_all_alignments':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_summary_all_alignments.xlsx')
    elif filetype == 'read_counts':
        return os.path.join(main_folder, 'results', N7 + '_' + N5 + '_read_counts.xlsx')

In [52]:
############################
#
# this was changed from the uditas original. It assumes I already went through the jupyternotebook of the
# normal uditias analysis. In short it assumes you already ran the correct_priming() function on the data
#
# This is used to trim the fastq files. Originally it was just to trim off the adapter sequences.
# However in this case we use it to trim off the sequences up to the break

# for Tn5 use the reverse compliment and read2. 

# ##########################
def trim_fastq_to_break(dir_sample, amplicon_info, seq_primer_to_breaksite):

    
    seqRC = reverse_complement(seq_primer_to_breaksite)
     
    
    # We first check if the experiment had any guides
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']

    #for Tn5 we only use read 1 as it is the sequence distal to the gene specific primer
    file_R1 = create_filename(dir_sample, N7, N5, 'R1fastq_CorrPrime')
    #file_R2 = create_filename(dir_sample, N7, N5, 'R2fastq_CorrPrime')
    
    # trimmedoutput files
    file_cutadapt_R1 = create_filename(dir_sample, N7, N5, 'R1trimmed2break')
    #file_cutadapt_R2 = create_filename(dir_sample, N7, N5, 'R2trimmed2break')
    file_cutadapt_report = create_filename(dir_sample, N7, N5, 'trimmed_report2break')
        
    if not os.path.exists(os.path.dirname(file_cutadapt_R1)):
        os.mkdir(os.path.dirname(file_cutadapt_R1))

    # remove adapters with cutadapt
    #original uditas peramiter had an error -e 0.33 (but was cutting of random stuff too much)
    cutadapt_command = ['cutadapt',
                        '-m', '20',
                        '-e', '0.20',
                        '-a', seqRC,
                        '-o', file_cutadapt_R1,
                        file_R1]

    handle_cutadapt_report = open(file_cutadapt_report, 'wb')
    subprocess.call(cutadapt_command, stdout=handle_cutadapt_report)
    handle_cutadapt_report.close()


In [53]:
trim_fastq_to_break(directory, amplicon_info, seq_primer_to_breaksite)

In [46]:
############################
#
# Aligns reads globally to amplicon. "end-to-end" as the default function in bowtie2
# Input: read trimmed up to the cut site
#        align to our reference amplicons to remove reads thats would align there
#
# ##########################
def align_afterbreak_reads_to_amplicons(dir_sample, amplicon_info, ncpu=4):

    # We first check if the experiment had any guides
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']

    #input file (for Tn5 we use R1 and LAM R2)
    file_R1 = create_filename(dir_sample, N7, N5, 'R1trimmed2break')
    #file_R2 = create_filename(dir_sample, N7, N5, 'R2trimmed2break')    

    if not os.path.exists(os.path.dirname(file_R1)):
        os.mkdir(os.path.dirname(file_R1))

    
    file_sam_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_sam_amplicons')
    file_sam_report_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_sam_report_amplicons')

    if not os.path.exists(os.path.dirname(file_sam_amplicons)):
        os.mkdir(os.path.dirname(file_sam_amplicons))

    file_bam_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_bam_amplicons')
    file_sorted_bam_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_sorted_bam_amplicons')

    if not os.path.exists(os.path.dirname(file_bam_amplicons)):
        os.mkdir(os.path.dirname(file_bam_amplicons))

    # global alignment to the amplicons with bowtie2
    initial_dir = os.getcwd()
    folder_amplicons = create_filename(dir_sample, N7, N5, 'amplicons')

    os.chdir(folder_amplicons)
    bowtie2_command = ['bowtie2', '-p', str(ncpu), '--very-sensitive',
                       '-X', '5000', '-k', '2', '-x', 'amplicons',
                       '-U', file_R1, '-S', file_sam_amplicons]

    handle_sam_report_amplicons = open(file_sam_report_amplicons, 'wb')

    subprocess.call(bowtie2_command, stderr=handle_sam_report_amplicons)

    handle_sam_report_amplicons.close()

    # convert sam to bam
    sam_to_bam_amplicons_command = ['samtools', 'view', '-Sb', file_sam_amplicons]

    handle_file_bam_amplicons = open(file_bam_amplicons, 'wb')

    subprocess.call(sam_to_bam_amplicons_command, stdout=handle_file_bam_amplicons)

    # sort bam files
    sort_bam_amplicons_command = ['samtools', 'sort', file_bam_amplicons, '-o', file_sorted_bam_amplicons]

    subprocess.call(sort_bam_amplicons_command)

    # Clean up
    #os.remove(file_sam_amplicons)
    os.remove(file_bam_amplicons)

    # Create bam index files
    create_bam_amplicons_index_command = ['samtools', 'index', file_sorted_bam_amplicons]
    subprocess.call(create_bam_amplicons_index_command)

    os.chdir(initial_dir)

################################################################################
# Function to extract all unmapped reads to the amplicons
################################################################################
def extract_unmapped_afterbreak_to_amplicon_reads(dir_sample, amplicon_info):

    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']

    file_sorted_bam_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_sorted_bam_amplicons')

    file_unmapped_bam_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_unmapped_bam_amplicons')

    file_qsorted_unmapped_bam_amplicons = create_filename(dir_sample, N7, N5, 'break_trimmed_qsorted_unmapped_bam_amplicons')

    file_R1_unmapped = create_filename(dir_sample, N7, N5, 'break_trimmed_unmapped_amplicons_R1fastq')
    file#_R2_unmapped = create_filename(dir_sample, N7, N5, 'unmapped_amplicons_R2fastq')
    file_unmapped_report = create_filename(dir_sample, N7, N5, 'break_trimmed_unmapped_amplicons_report')

    if not os.path.exists(os.path.dirname(file_R1_unmapped)):
        os.mkdir(os.path.dirname(file_R1_unmapped))

    extract_unmapped_bam_command = ['samtools', 'view', '-b', '-f', '0x4', file_sorted_bam_amplicons, '-o',
                                    file_unmapped_bam_amplicons]

    subprocess.call(extract_unmapped_bam_command)

    qsort_unmapped_bam_command = ['samtools', 'sort', '-n', file_unmapped_bam_amplicons, '-o',
                                  file_qsorted_unmapped_bam_amplicons]

    subprocess.call(qsort_unmapped_bam_command)

    bamtofastq_command = ['bedtools', 'bamtofastq', '-i', file_qsorted_unmapped_bam_amplicons,
                          '-fq', file_R1_unmapped]

    handle_unmapped_report = open(file_unmapped_report, 'wb')
    subprocess.call(bamtofastq_command, stderr=handle_unmapped_report)

    for fo in [file_R1_unmapped]:
        with open(fo) as f_in, gzip.open(fo + '.gz', 'wb') as f_out:
            f_out.writelines(f_in)
        os.remove(fo)

In [54]:
align_afterbreak_reads_to_amplicons(directory, amplicon_info)

In [55]:
extract_unmapped_afterbreak_to_amplicon_reads(directory, amplicon_info)

In [80]:
############################
#
# Align reads that did not map to the amplicons of interest
#
# Input: directory to be analyzed
#        amplicon_info, slice of sample_info.csv for the sample being processed
#        assembly, name of the assembly to be used by bowtie2. It is convenient to place it in a folder especified by
#        the environmental variable BOWTIE2_INDEXES
#
# ##########################
def align_afterbreak_genome_global(dir_sample, amplicon_info, assembly, ncpu=4):

    # We first check if the experiment had any guides
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    
    #input
    # Read1 for Tn5 Read2 for LAM
    file_R1 = create_filename(dir_sample, N7, N5, 'break_trimmed_unmapped_amplicons_R1fastqgz')
    #file_R2 = create_filename(dir_sample, N7, N5, 'unmapped_amplicons_R2fastqgz')

    file_sam_genome_global = create_filename(dir_sample, N7, N5, 'break_trimmed_sam_genome_global')
    file_sam_report_genome_global = create_filename(dir_sample, N7, N5, 'break_trimmed_sam_report_genome_global')

    if not os.path.exists(os.path.dirname(file_sam_genome_global)):
        os.mkdir(os.path.dirname(file_sam_genome_global))

    file_bam_genome_global = create_filename(dir_sample, N7, N5, 'break_trimmed_bam_genome_global')
    file_sorted_bam_genome_global = create_filename(dir_sample, N7, N5, 'break_trimmed_sorted_bam_genome_global')

    if not os.path.exists(os.path.dirname(file_bam_genome_global)):
        os.mkdir(os.path.dirname(file_bam_genome_global))

    # global alignment to the genome with bowtie2
    initial_dir = os.getcwd()

    bowtie2_command = ['bowtie2', '--very-sensitive', '-p', str(ncpu),
                       '-X', '5000', '-k', '2', '-x', assembly,
                       '-U', file_R1, '-S', file_sam_genome_global]

    handle_sam_report_genome_global = open(file_sam_report_genome_global, 'wb')

    subprocess.call(bowtie2_command, stderr=handle_sam_report_genome_global)

    handle_sam_report_genome_global.close()

    # convert sam to bam
    sam_to_bam_genome_global_command = ['samtools', 'view', '-Sb', file_sam_genome_global]

    handle_file_bam_genome_global = open(file_bam_genome_global, 'wb')

    subprocess.call(sam_to_bam_genome_global_command, stdout=handle_file_bam_genome_global)

    # sort bam files
    sort_bam_genome_global_command = ['samtools', 'sort', file_bam_genome_global, '-o', file_sorted_bam_genome_global]

    subprocess.call(sort_bam_genome_global_command)

    # Clean up
    #os.remove(file_sam_genome_global)
    os.remove(file_bam_genome_global)

    # Create bam index files
    create_bam_genome_global_index_command = ['samtools', 'index', file_sorted_bam_genome_global]
    subprocess.call(create_bam_genome_global_index_command)

    os.chdir(initial_dir)

    


############################
#       If you put in unmapped = 1 then it uses the unmapped to amplicons read, otherwise it just maps from trimmed fastq
# Aligns reads to the whole genome using bowtie2 and local alignment. This is needed to find translocations using
#   split reads and a program like socrates
#   --local aligment means that the sequence can be soft clipped at ends and is used in translocation programs that then align the clipped ends.
#
# Input: directory to be analyzed
#        amplicon_info, slice of sample_info.csv for the sample being processed
#        assembly, name of the assembly to be used by bowtie2. It is convenient to place it in a folder especified by
#        the environmental variable BOWTIE2_INDEXES
#
# ##########################
def align_afterbreaks_genome_local(dir_sample, unmapped_amplicons, amplicon_info, assembly):

    # We first check if the experiment had any guides
    N7 = amplicon_info['index_I1']
    N5 = amplicon_info['index_I2']
    
    #input
    # Read1 for Tn5 Read2 for LAM
    
    #If we want the trimmed ones
    if unmapped_amplicons == 1:
        file_R1 = create_filename(dir_sample, N7, N5, 'break_trimmed_unmapped_amplicons_R1fastqgz')
        #file_R2 = create_filename(dir_sample, N7, N5, 'unmapped_amplicons_R2fastqgz')

    else:
        #file_R1 = create_filename(dir_sample, N7, N5, 'R1trimmed2break')
        #file_R2 = create_filename(dir_sample, N7, N5, 'R2trimmed2break')

    file_sam_genome_local = create_filename(dir_sample, N7, N5, 'break_trimmed_sam_genome_local')
    file_sam_report_genome_local = create_filename(dir_sample, N7, N5, 'break_trimmed_sam_report_genome_local')

    if not os.path.exists(os.path.dirname(file_sam_genome_local)):
        os.mkdir(os.path.dirname(file_sam_genome_local))

    file_bam_genome_local = create_filename(dir_sample, N7, N5, 'break_trimmed_bam_genome_local')
    file_sorted_bam_genome_local = create_filename(dir_sample, N7, N5, 'break_trimmed_sorted_bam_genome_local')
    # file_sorted_bai_genome_local = create_filename(dir_sample, N7, N5, 'sorted_bai_genome_local')

    if not os.path.exists(os.path.dirname(file_bam_genome_local)):
        os.mkdir(os.path.dirname(file_bam_genome_local))

    # local alignment to the genome with bowtie2
    initial_dir = os.getcwd()

    bowtie2_command = ['bowtie2', '--local', '-p', str(ncpu),
                       '-X', '5000', '-k', '2', '-x', assembly,
                             '-U', file_R1, '-S', file_sam_genome_local]

    handle_sam_report_genome_local = open(file_sam_report_genome_local, 'wb')

    subprocess.call(bowtie2_command, stderr=handle_sam_report_genome_local)

    handle_sam_report_genome_local.close()

    # convert sam to bam
    sam_to_bam_genome_local_command = ['samtools', 'view', '-Sb', file_sam_genome_local]

    handle_file_bam_genome_local = open(file_bam_genome_local, 'wb')

    subprocess.call(sam_to_bam_genome_local_command, stdout=handle_file_bam_genome_local)

    # sort bam files
    sort_bam_genome_local_command = ['samtools', 'sort', file_bam_genome_local, '-o', file_sorted_bam_genome_local]

    subprocess.call(sort_bam_genome_local_command)

    # Clean up
    #os.remove(file_sam_genome_local)
    os.remove(file_bam_genome_local)

    # Create bam index files
    create_bam_genome_local_index_command = ['samtools', 'index', file_sorted_bam_genome_local]
    subprocess.call(create_bam_genome_local_index_command)

    os.chdir(initial_dir)

IndentationError: expected an indented block (<ipython-input-80-bfa004870f1c>, line 102)

In [56]:
align_afterbreak_genome_global(directory, amplicon_info, assembly)

In [81]:
align_afterbreaks_genome_local(directory, 1, amplicon_info, assembly)